In [1]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key: ········


In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader

# Load and process the text
loader = TextLoader('state_of_the_union.txt')
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [4]:
import chromadb
from chromadb.config import Settings

chroma_settings = Settings(chroma_api_impl="chromadb.api.fastapi.FastAPI", chroma_server_host="chroma", chroma_server_http_port=8000)
print ("Chroma settings: ", chroma_settings, flush=True)

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embedding, collection_name="state_of_the_union_test", client_settings=chroma_settings)

Chroma settings:  environment='' chroma_db_impl=None chroma_api_impl='chromadb.api.fastapi.FastAPI' chroma_telemetry_impl='chromadb.telemetry.posthog.Posthog' chroma_sysdb_impl='chromadb.db.impl.sqlite.SqliteDB' chroma_producer_impl='chromadb.db.impl.sqlite.SqliteDB' chroma_consumer_impl='chromadb.db.impl.sqlite.SqliteDB' chroma_segment_manager_impl='chromadb.segment.impl.manager.local.LocalSegmentManager' tenant_id='default' topic_namespace='default' is_persistent=False persist_directory='./chroma' chroma_server_host='chroma' chroma_server_headers=None chroma_server_http_port='8000' chroma_server_ssl_enabled=False chroma_server_grpc_port=None chroma_server_cors_allow_origins=[] anonymized_telemetry=True allow_reset=False migrations='apply'


In [5]:
vectordb.persist()
vectordb = None

In [6]:
vectordb = Chroma(collection_name="state_of_the_union_test", client_settings=chroma_settings,embedding_function=embedding)

In [7]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
